In [1]:
import tensorflow as tf
import os

from models import generator, discriminator, flownet, initialize_flownet
from loss_functions import intensity_loss, gradient_loss
from v_utils import DataLoader, load, save, psnr_error

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
dataset_name = 'vessel'
train_folder = '/dmount/Vessel/drawing_anno/Patient/train/image'
train_gtfolder = '/dmount/Vessel/drawing_anno/Patient/train/label'
test_folder = '/dmount/Vessel/drawing_anno/Patient/vessel/30_2_in'
test_gtfolder = '/dmount/Vessel/drawing_anno/Patient/vessel/30_2_out'

batch_size = 4
iterations = 10000
height, width = 256, 256


l_num = 2
alpha_num = 1
lam_lp = 1.0
lam_gdl = 1.0
num_his = 1

summary_dir = 'v_summary'
snapshot_dir = 'v_snapshot'

lr_bounds = [7000]
lr = [0.0001, 1e-05]

In [3]:
# define dataset
with tf.name_scope('dataset'):
    train_loader = DataLoader(train_folder, train_gtfolder, resize_h=height, resize_w=width)
    train_dataset = train_loader(batch_size=batch_size)

    train_it = train_dataset.make_one_shot_iterator()
    train_data = train_it.get_next()
    #train_data = train_data.set_shape([batch_size, height, width, 6])

    train_inputs = train_data[...,:3]
    train_gt = train_data[...,3:]

    print('train inputs = {}'.format(train_inputs))
    print('train prediction gt = {}'.format(train_gt))

    
    test_loader = DataLoader(test_folder, test_gtfolder, resize_h=height, resize_w=width)
    test_dataset = test_loader(batch_size=batch_size)
    
    test_it = test_dataset.make_one_shot_iterator()
    test_data = test_it.get_next()
    #test_data = test_data.set_shape([batch_size, height, width, 6])

    test_inputs = test_data[...,:3]
    test_gt = test_data[...,3:]

    print('test inputs = {}'.format(test_inputs))
    print('test prediction gt = {}'.format(test_gt))

# define training generator function
with tf.variable_scope('generator', reuse=None):
    print('training = {}'.format(tf.get_variable_scope().name))
    train_outputs = generator(train_inputs, layers=4, output_channel=3)
    train_psnr_error = psnr_error(gen_frames=train_outputs, gt_frames=train_gt)

# define testing generator function
with tf.variable_scope('generator', reuse=True):
    print('testing = {}'.format(tf.get_variable_scope().name))
    test_outputs = generator(test_inputs, layers=4, output_channel=3)
    test_psnr_error = psnr_error(gen_frames=test_outputs, gt_frames=test_gt)


# define intensity loss
if lam_lp != 0:
    lp_loss = intensity_loss(gen_frames=train_outputs, gt_frames=train_gt, l_num=l_num)
else:
    lp_loss = tf.constant(0.0, dtype=tf.float32)


# define gdl loss
if lam_gdl != 0:
    gdl_loss = gradient_loss(gen_frames=train_outputs, gt_frames=train_gt, alpha=alpha_num)
else:
    gdl_loss = tf.constant(0.0, dtype=tf.float32)


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
train inputs = Tensor("dataset/strided_slice:0", shape=(?, 256, 256, 3), dtype=float32)
train prediction gt = Tensor("dataset/strided_slice_1:0", shape=(?, 256, 256, 3), dtype=float32)
test inputs = Tensor("dataset/strided_slice_2:0", shape=(?, 256, 256, 3), dtype=float32)
test prediction gt = Tensor("dataset/strided_slice_3:0", shape=(?, 256, 256, 3), dtype=float32)
training = generator
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.cast` instead.

testing = generator


In [4]:
with tf.name_scope('training'):
    g_loss = tf.add_n([lp_loss * lam_lp, gdl_loss * lam_gdl], name='g_loss')

    g_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='g_step')
    g_lrate = tf.train.piecewise_constant(g_step, boundaries=lr_bounds, values=lr)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=g_lrate, name='g_optimizer')
    g_vars = tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

    g_train_op = g_optimizer.minimize(g_loss, global_step=g_step, var_list=g_vars, name='g_train_op')


# add all to summaries
tf.summary.scalar(tensor=train_psnr_error, name='train_psnr_error')
tf.summary.scalar(tensor=test_psnr_error, name='test_psnr_error')
tf.summary.scalar(tensor=g_loss, name='g_loss')
tf.summary.image(tensor=train_outputs, name='train_outputs')
tf.summary.image(tensor=train_gt, name='train_gt')
tf.summary.image(tensor=test_outputs, name='test_outputs')
tf.summary.image(tensor=test_gt, name='test_gt')
summary_op = tf.summary.merge_all()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # summaries
    summary_writer = tf.summary.FileWriter(summary_dir, graph=sess.graph)

    # initialize weights
    sess.run(tf.global_variables_initializer())
    print('Init successfully!')


    # tf saver
    saver = tf.train.Saver(var_list=tf.global_variables(), max_to_keep=None)
    restore_var = [v for v in tf.global_variables()]
    loader = tf.train.Saver(var_list=restore_var)
    if os.path.isdir(snapshot_dir):
        ckpt = tf.train.get_checkpoint_state(snapshot_dir)
        if ckpt and ckpt.model_checkpoint_path:
            load(loader, sess, ckpt.model_checkpoint_path)
        else:
            print('No checkpoint file found.')
    else:
        load(loader, sess, snapshot_dir)

    _step, _loss, _summaries = 0, None, None
    while _step < iterations:
        try:

            print('Training generator...')
            _, _g_lr, _step, _lp_loss, _gdl_loss, _g_loss, _train_psnr, _summaries = sess.run(
                [g_train_op, g_lrate, g_step, lp_loss, gdl_loss, g_loss, train_psnr_error, summary_op])

            if _step % 10 == 0:
                print('GeneratorModel : Step {}, lr = {:.6f}'.format(_step, _g_lr))
                print('                 Global      Loss : ', _g_loss)
                print('                 intensity   Loss : ({:.4f} * {:.4f} = {:.4f})'.format(_lp_loss, lam_lp, _lp_loss * lam_lp))
                print('                 gradient    Loss : ({:.4f} * {:.4f} = {:.4f})'.format( _gdl_loss, lam_gdl, _gdl_loss * lam_gdl))
                print('                 PSNR  Error      : ', _train_psnr)
            if _step % 100 == 0:
                summary_writer.add_summary(_summaries, global_step=_step)
                print('Save summaries...')

            if _step % 1000 == 0:
                save(saver, sess, snapshot_dir, _step)

        except tf.errors.OutOfRangeError:
            print('Finish successfully!')
            save(saver, sess, snapshot_dir, _step)
            break


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Init successfully!
No checkpoint file found.
Training generator...
Training generator...
Training generator...
Training generator...
Training generator...
Finish successfully!
INFO:tensorflow:v_snapshot/model.ckpt-4 is not in all_model_checkpoint_paths. Manually adding it.
The checkpoint has been created.
